In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

In [6]:
df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51182_2024-03-28~2025-03-28 (2).xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 37 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   결제 번호     196 non-null    object        
 1   상품 번호     196 non-null    int64         
 2   주문 번호     196 non-null    object        
 3   결제 완료일    196 non-null    datetime64[ns]
 4   주문 상태     196 non-null    object        
 5   상품명       196 non-null    object        
 6   옵션명       196 non-null    object        
 7   수량        196 non-null    int64         
 8   정상가       196 non-null    int64         
 9   판매가       196 non-null    int64         
 10  쿠폰 할인     196 non-null    int64         
 11  포인트 할인    196 non-null    int64         
 12  적립금       196 non-null    int64         
 13  정산가       196 non-null    int64         
 14  수령인명      196 non-null    object        
 15  수령인 연락처   196 non-null    object        
 16  구매자명      196 non-null    object        
 17  구매자 ID    196 no

In [8]:
df['주문 상태'].unique()
df = df[df['결제 완료일'] >= datetime(2025, 3, 19)]

In [10]:
df = df[df['주문 상태'].isin(['결제완료', '배송중', '배송완료'])]

In [12]:
daily_sales = pd.DataFrame(df.groupby(pd.Grouper(key='결제 완료일', freq='D'))['정산가'].sum())
daily_sales.columns = ['매출']
daily_sales

,매출
결제 완료일,
2025-03-25,13771900
2025-03-26,4464700
2025-03-27,2751700
2025-03-28,729700


In [142]:
df['date'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pd.pivot_table(df, index='date', columns='주문 상태', values='정산가', aggfunc='sum')

주문 상태,반품진행(반품진행),반품진행(수거진행),배송완료,배송중,취소완료
date,,,,,
2025-03-25,211600.0,114200.0,13771900.0,NaN,587800.0
2025-03-26,NaN,NaN,2785300.0,1891000.0,598700.0
2025-03-27,NaN,NaN,465200.0,2286500.0,114200.0
2025-03-28,NaN,NaN,NaN,289700.0,114200.0


In [132]:
temp_df = pd.pivot_table(df, index='date', columns='옵션명', values='정산가', aggfunc='sum', fill_value=0, margins=True, margins_name='총합')
temp_df.applymap(lambda x: f'{x:,}원')

옵션명,12박스(12개월분),3박스(3개월분),6박스(6개월분),총합
date,,,,
2025-03-25,"5,713,200원","1,777,700원","6,281,000원","13,771,900원"
2025-03-26,"1,269,600원","551,700원","2,855,000원","4,676,300원"
2025-03-27,"1,058,000원","551,700원","1,142,000원","2,751,700원"
2025-03-28,0원,"61,300원","228,400원","289,700원"
총합,"8,040,800원","2,942,400원","10,506,400원","21,489,600원"
